In [15]:
import sys
sys.path.append('/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/yolov5')
import torch
import os
import platform
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageOps

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadScreenshots, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_boxes, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.torch_utils import select_device, smart_inference_mode


In [16]:
device  = select_device('0')
device

YOLOv5 🚀 2022-9-28 Python-3.7.13 torch-1.12.1+cu102 CUDA:0 (Quadro RTX 5000, 16117MiB)



device(type='cuda', index=0)

In [17]:
weights = "/workspace/nabang1010/STEAM/DoQuocDang_STEAM/DoQuocDang_STEAM/src/yolov5/DoQuocDang_STEAM/yolov5x6_chest_xray_new/weights/best.pt"
data =  "/workspace/nabang1010/STEAM/DoQuocDang_STEAM/DoQuocDang_STEAM/src/yolov5/chest_xray_bg.yaml"


In [18]:
model = DetectMultiBackend(weights, device=device, data=data)
model.eval()

Fusing layers... 
Model summary: 416 layers, 140095828 parameters, 0 gradients, 208.2 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): 

In [19]:
df_efficientNet = pd.read_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/EfficientNetB4/submission.csv")
df_efficientNet

,image_id,abnormal
0,26a5285339df294ce288663b092c23c3,1
1,31ccf1b8a9992b055d0d26043eea3c30,1
2,688ecdb1a4e994d42b5a50a8c4a9736f,1
3,73a47bff27e977dc25c0b2cd4a470eea,1
4,8bb8636f45e172949bb99b3d5dd715fd,1
...,...,...
2995,f951f6a6d4fd2ac09f5e95a33c731a38,0
2996,3075e85dda12262bc9e34a5bd518d226,1
2997,40fb4c33b5cd086ae3eb5a38a6079fa2,0
2998,9575166c507d7862f5924bc606eb6c1d,1


In [20]:
df_efficientNet_abnormal = df_efficientNet[df_efficientNet['abnormal'] == 1]
df_efficientNet_abnormal

,image_id,abnormal
0,26a5285339df294ce288663b092c23c3,1
1,31ccf1b8a9992b055d0d26043eea3c30,1
2,688ecdb1a4e994d42b5a50a8c4a9736f,1
3,73a47bff27e977dc25c0b2cd4a470eea,1
4,8bb8636f45e172949bb99b3d5dd715fd,1
...,...,...
2992,558b20219d89e3918384de0ffffaec60,1
2994,9e03f33a5f100880b2d2bdfbc46b7b72,1
2996,3075e85dda12262bc9e34a5bd518d226,1
2998,9575166c507d7862f5924bc606eb6c1d,1


for c

In [21]:
stride, names, pt = model.stride, model.names, model.pt
vid_stride = 1
source = "/workspace/nabang1010/STEAM/DoQuocDang_STEAM/DoQuocDang_STEAM/DATA/test_png"

img_size = (640, 640)
dataset = LoadImages(source, img_size=img_size, stride=stride, auto=pt, vid_stride=vid_stride)
DF_submissions_final = pd.DataFrame(columns=['image_id', 'PredictionString'])
for path, im, im0s, vid_cap, s in  dataset: # chay het 3000 anh
    # print("=====================")
    image_name = path.split("/")[-1][0:-4]
    # check if image is in efficientNet abnormal
    submission_string = ""
    if image_name in df_efficientNet_abnormal['image_id'].values:  # neu co trong danh sach anh co abnormal
        
        img = torch.from_numpy(im).to(device).float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img[None]
        pred = model(img, augment=False)
        pred = non_max_suppression(pred, 0.5, 0.3, classes=None, agnostic=False)   # cho vao phat hien vung bat thuong
        # print(path)
        # print image name
        # print(image_name)
        for i, det in enumerate(pred):
            if len(det):                                                            # neu co vung bat thuong
                p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]] 
                det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()
                
                
                for *xyxy, conf, cls in reversed(det):
                    # xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()
                    label = f'{names[int(cls)]} {conf:.2f}'
                    class_id = int(cls)
                    # convert to coordinates in original image
                    x1 = int(xyxy[0].item())
                    y1 = int(xyxy[1].item())
                    x2 = int(xyxy[2].item())
                    y2 = int(xyxy[3].item())
                    # conf to string
                    conf = str(conf.item())
                    # print(image_name, class_id, label, x1, x2, y1, y2, conf)
                    submission_string += str(class_id) + " "+ conf + " "  + str(x1) + " " + str(y1) + " " + str(x2) + " " + str(y2) + " "
            else:   # neu khong co vung bat thuong
                submission_string += "14 1.0 0 0 1 1"

    else:   # neu khong co trong danh sach anh co abnormal
        submission_string += "14 1.0 0 0 1 1"
    DF_submissions_final = DF_submissions_final.append({'image_id': image_name, 'PredictionString': submission_string}, ignore_index=True)
                
DF_submissions_final.to_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/csv/submission_yolov5_14_bg_04_efficientNet.csv", index=False)